In [1]:
import os
import zipfile
import os
import json

from azureml.core.authentication import InteractiveLoginAuthentication, MsiAuthentication
from azureml.core import Workspace
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core.webservice import Webservice, AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.exceptions import WebserviceException

In [2]:
os.chdir('..')

In [3]:
# Custom functions
import sys
sys.path.append('./code')
import helper as he
import data as dt
import custom as cu

In [4]:
"""
PREPARE FOR DEPLOYMENT

NOTE: set language in custom.py before running

Example (in the command line):
> cd to root dir
> conda activate nlp
> python code/aml_deploy.py

TODO:
- fetch flair model if not found
"""



logger = he.get_logger(location=__name__)

language = 'en'
env = 'test'
version = '0.3'
upload = False

##############################
## CONNECT TO WORKSPACE
##############################

# try:
#     auth = MsiAuthentication()
# except Exception as e:
#     logger.warn(e)
auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")

ws = Workspace.get(name='nlp-ml', 
                subscription_id='50324bce-875f-4a7b-9d3c-0e33679f5d72', 
                resource_group='nlp', 
                auth=auth)

# ws = Workspace.from_config()
logger.warn(f'[INFO] Loaded Workspace {ws.name} ({ws.resource_group})')

#NOTE:
# Change language setting in custom.py before deployment!

model_name = f'nlp_{language}_{env}'
if upload:
    logger.warn(f'[INFO] Uploading model assets -> {model_name}')

    # Zip Assets
    dt_assets = dt.Data()
    with zipfile.ZipFile(dt_assets.fn_lookup['fn_asset'], 'w') as z:
        for f in cu.params.get('assets'):
            fp = dt_assets.fn_lookup[f]
            # print(fp)
            if '.' in fp:
                _new_fn = fp.split('/')[-1]
                z.write(fp, arcname=_new_fn)
                print(f'Zipped : {fp} -> {_new_fn}')
            else:
                # Iterate over all the files in directory
                for folderName, subfolders, filenames in os.walk(fp):
                    for filename in filenames:
                        filePath = os.path.join(folderName, filename)
                        _new_fn = folderName.split('/')[-1] + '/' + filename
                        z.write(filePath, arcname=_new_fn)
                        print(f'Zipped : {filePath} -> {_new_fn}')
    
    # Upload Assets
    model = Model.register(workspace=ws,
                        model_name=model_name,                      # Name of the registered model in your workspace.
                        model_path=dt_assets.fn_lookup['fn_asset'], # Local file to upload and register as a model.
                        #    resource_configuration=ResourceConfiguration(cpu=2, memory_in_gb=4),
                        description='NLP inference assets',
                        tags={'models': 'classification, ner, qa', 
                                'version': version,
                                'language': language, 
                                'environment': env})
else:
    logger.warn(f'[INFO] Using existing model -> {model_name}')
    model = Model(ws, model_name)

logger.warn(f'[INFO] \t Model Name: {model.name}')
logger.warn(f'[INFO] \t Model Version: {model.version}')

##############################
## DEPLOY COMPUTE
##############################
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4) #, auth_enabled=True)

environment = Environment('farmenv')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
                                                                                'azureml-defaults',
                                                                                'spacy',
                                                                                'transformers==2.3.0',
                                                                                'scipy',
                                                                                'numpy',
                                                                                'azure-storage-blob',
                                                                                'tqdm',
                                                                                'boto3',
                                                                                'scipy>=1.3.2',
                                                                                'sklearn',
                                                                                'seqeval',
                                                                                'mlflow==1.0.0',
                                                                                'dotmap==1.3.0',
                                                                                'git+https://github.com/deepset-ai/FARM.git',
                                                                                'git+https://github.com/zalandoresearch/flair.git'
                                                                            ],
                                                                 conda_packages=[
                                                                                'pytorch',
                                                                                'torchvision',
                                                                                'gensim',
                                                                                'numpy',
                                                                                'pandas'
                                                                            ])

service_name = f'nlp-{language}-{env}'

inference_config = InferenceConfig(entry_script='code/infer.py',
                                   source_directory='.',
                                   environment=environment)

# Remove any existing service under the same name.
try:
    #Webservice(ws, service_name).delete()
    # Retrieve existing service.
    service = Webservice(name=service_name, workspace=ws)
    logger.warn('[INFO] Updating web service')
    # Update to new model(s).
    service.update(models=[model], inference_config=inference_config)
    service.wait_for_deployment(show_output=True)
except WebserviceException:
    logger.warn('[INFO] Creating web service')
    service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config) #, overwrite=True)
    service.wait_for_deployment(show_output=True)
logger.warn(service.get_keys)

try:
    service.run(json.dumps([{"body": "Mein Windows Vista rechner will nicht mehr - ich kriege dauernd fehler meldungen. Ich wollte mir eh einen neuen kaufen, aber ich hab kein Geld. Kann Bill Gates mir helfen?"}]))
except Exception as e:
    logger.warn('[ERROR] Service was not deployed as expected. {e}')

C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
WARNING - [INFO] Loaded Workspace nlp-ml (nlp)
C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:79: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
WARNING - [INFO] Using existing model -> nlp_en_test
C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:82: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
WARNING - [INFO] 	 Model Name: nlp_en_test
C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:83: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
WARNING - [INFO] 	 Model Version: 4
C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:

Traceback (most recent call last):
  File "<ipython-input-4-8f8f27e0afb6>", line 126, in <module>
    service = Webservice(name=service_name, workspace=ws)
  File "C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\azureml\core\webservice\webservice.py", line 190, in __new__
    'workspace'.format(name))
azureml.exceptions._azureml_exception.WebserviceException: WebserviceException:
	Message: WebserviceNotFound: Webservice with name nlp-en-test not found in provided workspace
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "WebserviceNotFound: Webservice with name nlp-en-test not found in provided workspace"
    }
}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\makayser\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-in

TypeError: must be str, not list